In [17]:
import numpy as np
from scipy.stats import norm
import pyswarms as ps
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

def KernelDensityEstimator(data, bandwidth):
    kde = KernelDensity(bandwidth=bandwidth).fit(data[:, np.newaxis])
    return kde

""" def compute_ucv(kde_estimate, data):
    n = len(data)
    #operande1 = 1 / (2*np.sqrt(np.pi)* )
    log_likelihood = np.sum(kde_estimate)
    ucv = -2 * log_likelihood + (2 / n)
    return ucv  """
def delta(X1,X2,h):
    return ((X1-X2) / h)**2

def compute_ucv(bandwidths,data):
    print("Compute ucv bandwiths: "+ str(bandwidths))
    n = len(data)
    d = len(bandwidths)
    prod_bandwiths = np.prod(bandwidths)
    operande1 = 1/ (2* np.sqrt(np.pi)**(d) * n**(2) * prod_bandwiths)
    sum1 = []
    for i in range(n):
        sum2 = []
        for j in range(n):
            if j == i : continue
            deltas = []
            for k in range(d):
                deltas.append((delta(data[i],data[j],bandwidths[k])))
            sum_deltas = np.sum(deltas)
            result = np.exp((-1/4)* sum_deltas) - ((2 * 2**(d/2)) * np.exp((-1/2)* sum_deltas))
            sum2.append(result)
        sum1.append(np.sum(sum2))
    operande2 =  1/ (2* np.sqrt(np.pi)**(d) * n * prod_bandwiths) * np.sum(sum1)

    final_result = operande1 + operande2
    print("Compute ucv final result: "+str(final_result))

    return final_result





# Define the objective function to be optimized
def objective_function(position, data):
    print(position)
    print("\n")
    ucv = compute_ucv(position, data)  # Compute UCV metric
    print("The UCV METRIC : {}".format(ucv))
    return ucv
    

# Define the bounds for the bandwidth parameter
lower_bound = 0 * np.ones(2)
upper_bound = 1 * np.ones(2)
bounds = (lower_bound, upper_bound)

# Generate some synthetic data for demonstration
np.random.seed(42)
data = np.random.normal(loc=0, scale=1, size=100)

""" # Define the initial best position and objective value
initial_best_position = np.array([1.0])  # Initial bandwidth value
initial_best_objective_value = objective_function(initial_best_position, data)

# Set the best objective value as an attribute of the objective function
objective_function.best_objective_value = initial_best_objective_value """

# Initialize the optimizer
optimizer = ps.single.GlobalBestPSO(n_particles=2, dimensions=2, options={'c1': 0.5, 'c2': 0.3, 'w': 0.9},bounds=bounds)


# Perform the optimization
best_objective_value , best_positions  = optimizer.optimize(objective_function, iters=2, data=data)
print("\n")
for best_position in best_positions :
    print("Best bandwidth:", best_position)
print("Best UCV value:", best_objective_value)

2023-06-06 19:15:58,348 - pyswarms.single.global_best - INFO - Optimize for 2 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/2

[[0.3636296  0.97178208]
 [0.96244729 0.2517823 ]]


Compute ucv bandwiths: [[0.3636296  0.97178208]
 [0.96244729 0.2517823 ]]


pyswarms.single.global_best:  50%|█████     |1/2, best_cost=-72.5

Compute ucv final result: -72.46408860736783
The UCV METRIC : -72.46408860736783
[[0.81115326 0.24257256]
 [0.19277406 0.39070231]]


Compute ucv bandwiths: [[0.81115326 0.24257256]
 [0.19277406 0.39070231]]


pyswarms.single.global_best: 100%|██████████|2/2, best_cost=-297 
2023-06-06 19:15:58,876 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -297.1411622712238, best pos: [0.81115326 0.24257256]


Compute ucv final result: -297.1411622712238
The UCV METRIC : -297.1411622712238


Best bandwidth: 0.811153257682441
Best bandwidth: 0.24257256155605345
Best UCV value: -297.1411622712238
